In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
from tqdm import tqdm
HOME = expanduser("~")
import os
import cv2

In [2]:
#DIR = os.path.join(HOME, 'programming', 'dk39')
DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39'
MASKS = os.path.join(DIR, 'preps', 'masked')
CLEANED = os.path.join(DIR, 'preps', 'CH2', 'cleaned')
INPUT = os.path.join(DIR, 'preps', 'CH2', 'input')
OUTPUT = CLEANED
files = sorted(os.listdir(INPUT))
masks = sorted(os.listdir(MASKS))

In [3]:
def place_image(img, max_width, max_height):
    zmidr = max_height // 2
    zmidc = max_width // 2
    startr = zmidr - (img.shape[0] // 2)
    endr = startr + img.shape[0]
    startc = zmidc - (img.shape[1] // 2)
    endc = startc + img.shape[1]
    new_img = np.zeros([max_height, max_width])
    try:
        new_img[startr:endr,startc:endc] = img
    except:
        print('could not create new img', file, img.shape)
        
    return new_img.astype('uint16')


print(len(files), len(masks))

469 469


In [4]:
dels = os.listdir(OUTPUT)
for d in dels:
    os.unlink(os.path.join(OUTPUT, d))

In [5]:
# get oriented for comparison
max_width = 1750
max_height = 1050

img_inputs = []
img_outputs = []
file_inputs = []
for i, file in enumerate(tqdm(files)):
    infile = os.path.join(INPUT, file)
    maskfile = os.path.join(MASKS, file)
    src = io.imread(infile)
    mask = io.imread(maskfile)
    mask16 = np.copy(mask).astype('uint16')
    mask16[mask16 > 0] = 2**16-1
    #img_inputs.append(src)
    #file_inputs.append(file)
    fixed = cv2.bitwise_and(src, mask16)
    #img_outputs.append(fixed)
    fixed = place_image(fixed, max_width, max_height)
    outpath = os.path.join(OUTPUT, file)    
    cv2.imwrite(outpath, fixed.astype('uint16'))
print('Finished')


100%|██████████| 469/469 [00:22<00:00, 20.69it/s]

Finished


In [6]:
# display images
# check alignment
start, finish = (0,1)
displays = img_outputs[start:finish]
names = file_inputs[start:finish]
plt.style.use('classic')
for name,img in zip(names,displays):    
    plt.figure()
    plt.title('{}'.format(name))
    #plt.title('{}'.format(name))
    plt.imshow(img, cmap='gray')
    plt.show()